In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, TimeSeriesSplit, KFold, RepeatedKFold, \
                                    train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(color_codes=True)
from sklearn.metrics import precision_score, mean_squared_error, r2_score, make_scorer, adjusted_rand_score, \
                    accuracy_score, f1_score, confusion_matrix, classification_report, roc_auc_score, recall_score
from time import time
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import quantile_transform
import scipy.stats as st
from sklearn.feature_selection import RFE, RFECV, SelectFromModel
from sklearn.ensemble import RandomForestRegressor
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
import pprint as pp
import datetime
%matplotlib inline

In [ ]:
Climate_Data = pd.read_excel('Climate_Data.xls')
#######################################################################################################################
Energy_Data_mean = Climate_Data.groupby(['Year', 'Month', 'Day of Month']).mean()
Energy_Data_mean = Energy_Data_mean[['Day of Week', 'Is Holiday', 'Daylight Savings', 'DHI', 'DNI', 'Dew Point', 
                                     'Temperature', 'Relative Humidity']]
Energy_Data_mean.columns = ['Day_of_Week', 'Is_Holiday', 'Daylight_Savings', 'DHI_AVG', 'DNI_AVG', 'Dew Point_AVG', 
                            'Temperature_AVG', 'Relative Humidity_AVG']
#######################################################################################################################
Energy_Data_sum = Climate_Data.groupby(['Year', 'Month', 'Day of Month']).sum()
Energy_Data_sum = Energy_Data_sum[['DHI', 'DNI', 'Dew Point', 'Temperature', 'Relative Humidity']]
Energy_Data_sum.columns = ['DHI_SUM', 'DNI_SUM', 'Dew Point_SUM', 'Temperature_SUM', 'Relative Humidity_SUM']
#######################################################################################################################
Energy_Data_max = Climate_Data.groupby(['Year', 'Month', 'Day of Month']).max()
Energy_Data_max = Energy_Data_max[['DHI', 'DNI', 'Dew Point', 'Temperature', 'Relative Humidity']]
Energy_Data_max.columns = ['DHI_MAX', 'DNI_MAX', 'Dew Point_MAX', 'Temperature_MAX', 'Relative Humidity_MAX']
#######################################################################################################################
Energy_Data_std = Climate_Data.groupby(['Year', 'Month', 'Day of Month']).std()
Energy_Data_std = Energy_Data_std[['DHI', 'DNI', 'Dew Point', 'Temperature', 'Relative Humidity']]
Energy_Data_std.columns = ['DHI_STD', 'DNI_STD', 'Dew Point_STD', 'Temperature_STD', 'Relative Humidity_STD']
#######################################################################################################################
Energy_Data_min = Climate_Data.groupby(['Year', 'Month', 'Day of Month']).min()
Energy_Data_min = Energy_Data_min[['DHI', 'DNI', 'Dew Point', 'Temperature', 'Relative Humidity']]
Energy_Data_min.columns = ['DHI_MIN', 'DNI_MIN', 'Dew Point_MIN', 'Temperature_MIN', 'Relative Humidity_MIN']
#######################################################################################################################
Energy_Data = pd.concat([Energy_Data_mean, Energy_Data_sum, Energy_Data_max, Energy_Data_std, Energy_Data_min], axis=1)
Energy_Data.reset_index(inplace=True)
Energy_Data[['Energy_Consumption', 'True_Labels']] = pd.read_excel('EnergyData_D1.xlsx')
#######################################################################################################################
Energy_Data['Lag1'] = (Energy_Data['Energy_Consumption'].shift(1))
Energy_Data.dropna(axis=0,inplace=True)
#######################################################################################################################
Energy_Data['Date_Time'] = pd.to_datetime(pd.DataFrame({'year': Energy_Data['Year'],'month': Energy_Data['Month'] + 1,
                                                        'day': Energy_Data['Day of Month']}))

In [ ]:
Feature_Names = ['Month','Day_of_Week', 'Is_Holiday', 'Daylight_Savings', 'DHI_AVG', 'DNI_AVG', 'Dew Point_AVG', 
                 'Temperature_AVG', 'Relative Humidity_AVG', 'DHI_SUM', 'DNI_SUM', 'Dew Point_SUM', 'Temperature_SUM', 
                 'Relative Humidity_SUM', 'DHI_MAX', 'DNI_MAX', 'Dew Point_MAX', 'Temperature_MAX', 
                 'Relative Humidity_MAX', 'DHI_STD', 'DNI_STD', 'Dew Point_STD', 'Temperature_STD', 
                 'Relative Humidity_STD', 'DHI_MIN', 'DNI_MIN', 'Dew Point_MIN', 'Temperature_MIN', 
                 'Relative Humidity_MIN', 'Lag1']

X = Energy_Data[Feature_Names].as_matrix()
y = Energy_Data['Energy_Consumption'].as_matrix()
True_Labels = Energy_Data['True_Labels'].as_matrix()
date_time = Energy_Data['Date_Time']

In [ ]:
#################################################################################################
# To test anomaly detector
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, shuffle=False)
TL_train, TL_Test = train_test_split(True_Labels, test_size=0.5, shuffle=False)
DT_train, DT_Test = train_test_split(date_time, test_size=0.5, shuffle=False)
#################################################################################################

In [ ]:
def calc_dyn_threshold(A, P, I, N):
    # Control false alarm rates by tuning I and N. eg. increase I or N to reduce false alarms
    threshold = np.zeros(I-1)
    threshold[0:(I-1)] = P[0:(I-1)]
    labels = np.zeros(I-1)
    for k in np.arange(I,len(P)+1):
        #print(k)
        mu = np.mean(P[(k-I):k])
        #mu = np.mean(P[0:k])
        #print(mu)
        sigma = np.std(P[(k-I):k])
        #sigma = np.std(P[0:k])
        #print(sigma)
        T = mu + N*sigma
        #print(T)
        threshold = np.append(threshold,T)
        #print(threshold)
        if (A[k-1] > threshold[k-1]) :
            labels = np.append(labels,1)
        else:
            labels = np.append(labels,0)
    #print(P, labels, threshold)
    return labels, threshold

In [ ]:
def anomalyDetector():
    t0 = time()
    np.random.seed(7)
    ########################################################################################
    # Regression
    kf = KFold(n_splits=5, shuffle=True)
    scoring_param = make_scorer(mean_squared_error,greater_is_better=False)
    
    rfecv = RFECV(estimator=RandomForestRegressor(n_jobs=-1), step=1, cv=kf, scoring=scoring_param)
    FS_model = rfecv.fit(X_train, y_train)
    
    ranks = FS_model.ranking_
    FN =[]
    for i in range(len(ranks)):
        if ranks[i] == 1:
            FN.append(Feature_Names[i])    
    print(FN)
    
    X = Energy_Data[FN].as_matrix()
    X_train_transformed, X_test_transformed = train_test_split(X, test_size=0.5, shuffle=False)
    
    NE = [int(i) for i in np.linspace(100,1000,num=10)]
    p_grid = dict()
    p_grid = dict(n_estimators = NE)
    
    model = GridSearchCV(estimator = RandomForestRegressor(n_jobs=-1), param_grid = p_grid, 
                         scoring = scoring_param, cv = kf)
    model.fit(X_train_transformed, y_train)
    
    params = model.best_params_
    print("Best Est: %s" % (params['n_estimators']))
    
    Y_Test_Pred = model.predict(X_test_transformed)
    
    rmse = np.sqrt(mean_squared_error(y_test,Y_Test_Pred))
    data_range = y_test.max() - y_test.min()
    NRMSE = (rmse/data_range) * 100.0
    RSQ = r2_score(y_test,Y_Test_Pred)
    print("Normalized RMSE: %0.3f" % NRMSE)
    print("R-squared: %0.3f" % RSQ)
    ########################################################################################
    # Calculate dynamic threshold
    
    #Testing
    Labels, Threshold = calc_dyn_threshold(y_test, Y_Test_Pred, 2, 2)
    Temp = pd.DataFrame(data={'Date_Time': DT_Test, 'Actual': y_test, 'Predicted':Y_Test_Pred, 'Labels':TL_Test, 
                               'Threshold':Threshold, 'Pred_Labels': Labels})
    Temp.sort_values(by=['Date_Time'],inplace=True)
    Temp = Temp[Temp['Date_Time'].between('2016-04-01','2016-11-15')]
    
    # Establishing the relationship between I, N, and model effectiveness
    I = np.arange(2,31)
    N = np.arange(1,4)
    scores = np.zeros((len(I)*len(N), 6))
    ind = 0
    for i in I:
        for n in N:
            Labels, Threshold = calc_dyn_threshold(np.array(Temp['Actual']), np.array(Temp['Predicted']), i, n)
            scores[ind,0] = i
            scores[ind,1] = n
            scores[ind,2] = roc_auc_score(np.array(Temp['Labels']), Labels)
            scores[ind,3] = precision_score(np.array(Temp['Labels']), Labels)
            scores[ind,4] = recall_score(np.array(Temp['Labels']), Labels)
            scores[ind,5] = f1_score(np.array(Temp['Labels']), Labels)
            ind = ind + 1
    best_i = scores[scores[:,2].argmax(),0]
    best_n = scores[scores[:,2].argmax(),1]
    print(best_i,best_n)
    
    print("########################################################################################")
    print("Confusion Matrix - testing:")
    print(confusion_matrix(Temp['Labels'], Temp['Pred_Labels']))
    tn, fp, fn, tp = confusion_matrix(Temp['Labels'], Temp['Pred_Labels']).ravel()
    print("True Negative, False Positive, False Negative, True Positive {}.".format([tn, fp, fn, tp]))
    print("False positive means false alarms")
    print("False Negative means missed faults")
    print("########################################################################################")
    print("Classification Report - testing:")
    print(classification_report(Temp['Labels'], Temp['Pred_Labels'], target_names=['Normal', 'Fault']))
    print("########################################################################################")
    print("Accuracy - testing: %0.3f" % accuracy_score(Temp['Labels'], Temp['Pred_Labels']))
    print("########################################################################################")
    print("ROC AUC score - testing: %0.3f" % roc_auc_score(Temp['Labels'], Temp['Pred_Labels']))
    print("########################################################################################")
    ########################################################################################
    
    fig = plt.figure(figsize=(25,20))
    ax = fig.add_subplot(1, 1, 1)
    Data_0 = Temp.loc[Temp['Labels'][Temp['Labels']==0].index]
    Data_1 = Temp.loc[Temp['Labels'][Temp['Labels']==1].index]
    ax.scatter(Data_0['Date_Time'].dt.to_pydatetime(), Data_0['Actual'], c=plt.cm.coolwarm(0.), s=200, 
               edgecolors='y', marker='o', label=u'Actual normal data')
    ax.scatter(Data_1['Date_Time'].dt.to_pydatetime(), Data_1['Actual'], c=plt.cm.coolwarm(1.), s=200, 
               edgecolors='y', marker='^', label=u'Actual fault data')
    plt.plot(Temp['Date_Time'].dt.to_pydatetime(), Temp['Predicted'], 'c-*', lw = 4, ms = 5, label=u'XGBoost Prediction')
    plt.xlabel('Date Time',fontsize=30)
    plt.ylabel('Energy Consumption [J]',fontsize=30)
    plt.xticks(fontsize=30)
    plt.yticks(fontsize=30)
    plt.legend(loc='best',fontsize=30)
    plt.savefig('M2-D1-Actual-Labels-Predictions')
    
    fig = plt.figure(figsize=(25,20))
    ax = fig.add_subplot(1, 1, 1)
    Data_0 = Temp.loc[Temp['Pred_Labels'][Temp['Pred_Labels']==0].index]
    Data_1 = Temp.loc[Temp['Pred_Labels'][Temp['Pred_Labels']==1].index]
    ax.scatter(Data_0['Date_Time'].dt.to_pydatetime(), Data_0['Actual'], c=plt.cm.coolwarm(0.), s=200, 
               edgecolors='y', marker='o', label=u'Predicted normal data')
    ax.scatter(Data_1['Date_Time'].dt.to_pydatetime(), Data_1['Actual'], c=plt.cm.coolwarm(1.), s=200, 
               edgecolors='y', marker='^', label=u'Predicted fault data')
    plt.plot(Temp['Date_Time'].dt.to_pydatetime(), Temp['Predicted'], 'c-*', lw = 4, ms = 5, label=u'XGBoost Prediction')
    plt.plot(Temp['Date_Time'].dt.to_pydatetime(), Temp['Threshold'], 'k--', lw = 4, label=u'Dynamic threshold')
    plt.xlabel('Date Time',fontsize=30)
    plt.ylabel('Energy Consumption [J]',fontsize=30)
    plt.xticks(fontsize=30)
    plt.yticks(fontsize=30)
    plt.legend(loc='best',fontsize=30)
    plt.savefig('M2-D1-RF-Dynamic-Threshold-Predicted-Labels')
    
    t1 = time()
    print("Time taken: %0.3f" % (t1-t0))
    return scores
    #########################################################################################

In [ ]:
scores = anomalyDetector()

In [ ]:
Temp = pd.DataFrame(data=scores,columns=['I','N','ROC_AUC', 'Precision', 'Recall', 'F1_Score'])
Data1 = Temp.loc[Temp['N'][Temp['N']==1].index]
Data2 = Temp.loc[Temp['N'][Temp['N']==2].index]
Data3 = Temp.loc[Temp['N'][Temp['N']==3].index]

In [ ]:
fig = plt.figure(figsize=(25,20))
ax = fig.add_subplot(1, 1, 1)
plt.plot(Data1['I'], Data1['F1_Score'], 'c--s', lw = 6, ms = 10, label=u'$\mathit{k}$=1')
plt.plot(Data2['I'], Data2['F1_Score'], 'r-o', lw = 6, ms = 10, label=u'$\mathit{k}$=2')
plt.plot(Data3['I'], Data3['F1_Score'], 'g--D', lw = 6, ms = 10, label=u'$\mathit{k}$=3')
plt.xlabel('$\mathit{j}$',fontsize=40)
plt.ylabel('F1 score',fontsize=40)
plt.xticks(fontsize=30)
plt.yticks(fontsize=30)
plt.legend(loc='best',fontsize=40)
plt.savefig('M1-D1-Relationship-J-K-F1')

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf

fig1 = plt.figure(figsize=(20,20))
ax1 = fig1.add_subplot(1, 1, 1)
plot_acf(y_train,lags=60,ax=ax1)
plt.xlabel('Lag', fontsize=40)
plt.ylabel('Autocorrelation', fontsize=40)
plt.xticks(fontsize=30)
plt.yticks(fontsize=30)
plt.figaspect(1.)
plt.title("",fontsize=20)
plt.savefig('Chiller-ACF')

In [ ]:
fig1 = plt.figure(figsize=(20,10))
ax0 = fig1.add_subplot(1, 1, 1)

sns.distplot(y_train, kde=False, color = plt.cm.coolwarm(0.), ax=ax0, norm_hist=True, bins=10)
ax0.set_ylabel('Probability',fontsize=40)
ax0.set_xlabel('Energy consumption',fontsize=40)

plt.savefig('Chiller-Data-Distribution')